<a href="https://colab.research.google.com/github/Flotapponnier/Recognition-of-my-face/blob/main/Facerecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Start program

### installing dependencies

In [ ]:
!pip install torch torchvision opencv-python matplotlib


### Import from drive data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

data_dir = "/content/drive/MyDrive/face_dataset"
my_face_path = os.path.join(data_dir, "my_face")
not_my_face_path = os.path.join(data_dir, "not_my_face")


### Start traiting image

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

### Dataset personnalised with Pytorch

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class FaceDataset(Dataset):
    def __init__(self, folder_path, transform):
        self.images = []
        self.labels = []
        self.transform = transform

        for label, folder in enumerate(["not_my_face", "my_face"]):
            folder_full = os.path.join(folder_path, folder)
            for img_name in os.listdir(folder_full):
                self.images.append(os.path.join(folder_full, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("RGB")
        return self.transform(img), self.labels[idx]


### Loading data




In [ ]:
from torch.utils.data import DataLoader, random_split

# Assure-toi que le dataset existe déjà
dataset = FaceDataset(data_dir, transform)

# Split en train et validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Création des dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


### Personnalised dataset pytorch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        # ➤ Ajoute une étape temporaire pour calculer la taille de sortie
        self._to_linear = None
        self._get_conv_output()

        self.fc1 = nn.Linear(self._to_linear, 64)
        self.fc2 = nn.Linear(64, 2)

    def _get_conv_output(self):
        x = torch.randn(1, 3, 128, 128)  # taille de tes images
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        self._to_linear = x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


### Train model

In [ ]:
import torch
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 25
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")



### Evaluate model

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")


Accuracy: 78.95%


In [ ]:
### Test on another picture

In [ ]:
def predict_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)
    output = model(img)
    _, predicted = torch.max(output, 1)
    return "Florent detected!" if predicted.item() == 1 else "Florent not detected"

# Exemple :
print(predict_image("/content/drive/MyDrive/face_dataset/test/testface3.png"))


### Save the model


In [ ]:
import shutil

torch.save(model.state_dict(), "face_model.pth")
shutil.copy("face_model.pth", "/content/drive/MyDrive/face_dataset/Models/face_model.pth")


'/content/drive/MyDrive/face_dataset/Models/face_model.pth'

## Load and test on false visage to test the model


### Charge Model

In [ ]:
model = SimpleCNN()
model.load_state_dict(torch.load("/content/drive/MyDrive/Models/face_model.pth"))
model.eval()

### Charge random visage



In [ ]:
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as transforms

def get_fake_face():
    url = "https://thispersondoesnotexist.com"
    response = requests.get(url, timeout=5)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    return image


### transform


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [ ]:
import torch

def is_my_face(img, threshold=0.5):
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=1)
        my_face_prob = probs[0][1].item()
    return my_face_prob >= threshold, my_face_prob

In [ ]:
import time
save_path = "/content/drive/MyDrive/face_dataset/face_recognize"
os.makedirs(save_path, exist_ok=True)

max_attempts = 5000
found = False

for i in range(max_attempts):
    img = get_fake_face()
    detected, prob = is_my_face(img)
    print(f"Essai {i+1} — Probabilité que ce soit toi : {prob:.4f}")

    if detected:
        print("🎉 Un visage est détecté comme le tien !")
        img.show()

        img_filename = f"{save_path}/detected_face_{i+1}_prob_{prob:.4f}.jpg"
        img.save(img_filename)
        print(f"💾 Image sauvegardée sous : {img_filename}")

        found = True

if not found:
    print("😢 Aucun visage détecté comme étant le tien après 50 essais.")


Essai 1 — Probabilité que ce soit toi : 0.0000
Essai 2 — Probabilité que ce soit toi : 0.0000
Essai 3 — Probabilité que ce soit toi : 0.0011
Essai 4 — Probabilité que ce soit toi : 0.0005
Essai 5 — Probabilité que ce soit toi : 0.0425
Essai 6 — Probabilité que ce soit toi : 0.0139
Essai 7 — Probabilité que ce soit toi : 0.0001
Essai 8 — Probabilité que ce soit toi : 0.0001
Essai 9 — Probabilité que ce soit toi : 0.0010
Essai 10 — Probabilité que ce soit toi : 0.0000
Essai 11 — Probabilité que ce soit toi : 0.0085
Essai 12 — Probabilité que ce soit toi : 0.0069
Essai 13 — Probabilité que ce soit toi : 0.0027
Essai 14 — Probabilité que ce soit toi : 0.0000
Essai 15 — Probabilité que ce soit toi : 0.0823
Essai 16 — Probabilité que ce soit toi : 0.0000
Essai 17 — Probabilité que ce soit toi : 0.0108
Essai 18 — Probabilité que ce soit toi : 0.0075
Essai 19 — Probabilité que ce soit toi : 0.0002
Essai 20 — Probabilité que ce soit toi : 0.0001
Essai 21 — Probabilité que ce soit toi : 0.0584
E

KeyboardInterrupt: 